In [33]:
#import dependencies 
from numpy.random import seed
from numpy.random import randn
from scipy import stats
import numpy as np

import math  

import seaborn as sns
sns.set(style="darkgrid")
#sns.set(style="whitegrid")
from matplotlib import pyplot as plt

import pandas as pd
import os

In [3]:
#check list of files in data 
directory = "./data/dedicated_host/"
print(os.listdir(directory))

['.DS_Store', 'm5d_pgbench_dedicated_host_11-16-2019.csv', 'm5d_y-cruncher_dedicated_host_11-16-2019_us-east-1 - Sheet1.csv', 'm5d_iperf_dedicated_host_11-17.csv', 'm5d_sysbench_dedicated_host_11-16-2019_us-east-1b - Sheet1.csv', 'newdata.csv']


In [4]:
#DON'T FORGET TO SET THESE!

#set experiment type, csv data, and dedicated host mode...
experiment_type = 'iperf' #SET TO ONE OF: sysbench, y-cruncher, pgbench, iperf
data_name = 'm5d_iperf_dedicated_host_11-17.csv'

dedicated_host_mode = True

In [5]:
df_data = pd.read_csv(directory + data_name, header = 0);

In [6]:
df_data.head()

,instanceID,instanceType,experimentID,interval,bandwidthUpload,bandwitdhDownload,setId,vmId,cmd
0,i-0ede59d8254436a40,m5d.large,1573987010,15.0 sec,4.97 Gbits/sec,4.45 Gbits/sec,5,5,iperf -c 172.31.40.155 --dualtest --window 416...
1,i-0ede59d8254436a40,m5d.large,1573987010,15.0 sec,4.86 Gbits/sec,4.62 Gbits/sec,5,5,iperf -c 172.31.40.155 --dualtest --window 416...
2,i-0ede59d8254436a40,m5d.large,1573987010,15.0 sec,4.97 Gbits/sec,4.63 Gbits/sec,5,5,iperf -c 172.31.40.155 --dualtest --window 416...
3,i-0ede59d8254436a40,m5d.large,1573987010,15.0 sec,4.95 Gbits/sec,4.52 Gbits/sec,5,5,iperf -c 172.31.40.155 --dualtest --window 416...
4,i-0ede59d8254436a40,m5d.large,1573987010,15.0 sec,4.95 Gbits/sec,2.71 Gbits/sec,5,5,iperf -c 172.31.40.155 --dualtest --window 416...


In [7]:
#iperf remove trailing text
if (experiment_type == 'iperf'):
    df_data['bandwidthUpload'] = df_data['bandwidthUpload'].str.split(" ").str[0].astype('float64')
    df_data['bandwitdhDownload'] = df_data['bandwitdhDownload'].str.split(" ").str[0].astype('float64')
    #df_data.dropna(inplace = True)

In [8]:
df_data.tail()

,instanceID,instanceType,experimentID,interval,bandwidthUpload,bandwitdhDownload,setId,vmId,cmd
12931,i-0ab84c959b473605a,m5d.large,1573987010,15.0 sec,437.00,456.00,48,40,iperf -c 172.31.42.49 --dualtest --window 416k...
12932,i-0ab84c959b473605a,m5d.large,1573987010,15.0 sec,1.43,1.63,48,40,iperf -c 172.31.42.49 --dualtest --window 416k...
12933,i-0ab84c959b473605a,m5d.large,1573987010,15.0 sec,899.00,1.31,48,40,iperf -c 172.31.42.49 --dualtest --window 416k...
12934,i-0ab84c959b473605a,m5d.large,1573987010,15.0 sec,1.27,1.40,48,40,iperf -c 172.31.42.49 --dualtest --window 416k...
12935,i-0ab84c959b473605a,m5d.large,1573987010,15.0 sec,652.00,1.54,48,40,iperf -c 172.31.42.49 --dualtest --window 416k...


In [9]:
if (experiment_type == 'iperf'):
    #adjust gigbite values 'up' assuming all mbs values are over 100mb
    df_data['bandwitdhDownload'] = np.where(df_data['bandwitdhDownload'] > 100, df_data['bandwitdhDownload'], df_data['bandwitdhDownload']*1000)
    df_data['bandwidthUpload'] = np.where(df_data['bandwidthUpload'] > 100, df_data['bandwidthUpload'], df_data['bandwidthUpload']*1000)
    
    #add total bandwidth column
    df_data['total'] = df_data['bandwidthUpload'] + df_data['bandwitdhDownload']
    
    #sort by setId, vmId
    df_data = df_data.sort_values(by =['setId', 'vmId'], ascending = False)
    
    #reset index
    df_data.reset_index(drop=True, inplace= True)

In [10]:
#set nth column #change depending on data set...
if (experiment_type == 'iperf'):
    nth_column = 11
    if (not dedicated_host_mode):
        df_data = df_data[:551] #drop every run except first run
elif (experiment_type != 'pgbench'):
    nth_column = 10

In [11]:
#drop every nth column
#drop first row of each vm run since they are outliers (except pgbench)
if (experiment_type != 'pgbench'):
    df_data = df_data[df_data.index % nth_column != 0]

In [12]:
#check if correct columns are dropped..
df_data.head(14)

,instanceID,instanceType,experimentID,interval,bandwidthUpload,bandwitdhDownload,setId,vmId,cmd,total
1,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,262.0,296.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,558.0
2,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,345.0,365.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,710.0
3,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,721.0,505.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,1226.0
4,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,380.0,348.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,728.0
5,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,513.0,410.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,923.0
6,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,502.0,415.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,917.0
7,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,2070.0,1220.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,3290.0
8,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,2130.0,1260.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,3390.0
9,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,1990.0,1200.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,3190.0
10,i-07c76ac1348426483,m5d.large,1573987010,15.0 sec,2920.0,1480.0,48,48,iperf -c 172.31.38.25 --dualtest --window 416k...,4400.0


In [13]:
#if in dedicated host mode, group by instanceID and 
if(dedicated_host_mode):
    if (experiment_type != 'iperf'):
        #create setId column
        df_data['setId'] = df_data['experimentID'].str.strip().str[-2:]
        df_data['setId'] = df_data['setId'].astype('int32').abs()
        
    df = df_data.groupby(['setId', 'instanceID']).mean()
else:
    #group by instanceID and take the mean of each column
    df = df_data.groupby('instanceID').mean()

In [14]:
#convert setId index to column
df['setId'] = df.index.get_level_values(0)

In [15]:
#add tenancy info
tenant_count = 48 #change to max vm count, relative to VM type, if necessary

if(experiment_type == 'iperf'):
    df['tenants'] = df['setId']
else:
    df['tenants'] = tenant_count - df['setId']

In [16]:
df.tail()

experimentID  bandwidthUpload  bandwitdhDownload  \
setId instanceID                                                              
48    i-0dcfa6d3bc470d28d    1573987010            490.5              626.5   
      i-0e167bbfcd6a685f5    1573987010            818.5              661.9   
      i-0ede59d8254436a40    1573987010            558.6              521.0   
      i-0f285214d5c41327e    1573987010            567.5              471.9   
      i-0f472084ac3992ad9    1573987010            735.0              767.6   

                           vmId   total  setId  tenants  
setId instanceID                                         
48    i-0dcfa6d3bc470d28d    30  1117.0     48       48  
      i-0e167bbfcd6a685f5    34  1480.4     48       48  
      i-0ede59d8254436a40     5  1079.6     48       48  
      i-0f285214d5c41327e    21  1039.4     48       48  
      i-0f472084ac3992ad9    36  1502.6     48       48

In [17]:
#add standard deviation
if (dedicated_host_mode):
    df_std = df_data.groupby(['setId', 'instanceID']).std()
else:
    df_std = df_data.groupby('experimentID').std()

In [18]:
df_std.head()

experimentID  bandwidthUpload  bandwitdhDownload  \
setId instanceID                                                              
1     i-06b96f53253b4cc2e           0.0              0.0         321.601893   
2     i-06b96f53253b4cc2e           0.0              0.0         100.421335   
      i-0d44050a4e6a71d7c           0.0              0.0         141.985915   
3     i-06b96f53253b4cc2e           0.0              0.0         747.580542   
      i-0c3e9f9e7b281edfe           0.0              0.0          64.807407   

                           vmId       total  
setId instanceID                             
1     i-06b96f53253b4cc2e   0.0  321.601893  
2     i-06b96f53253b4cc2e   0.0  100.421335  
      i-0d44050a4e6a71d7c   0.0  141.985915  
3     i-06b96f53253b4cc2e   0.0  747.580542  
      i-0c3e9f9e7b281edfe   0.0   64.807407

In [19]:
#drop a single index level
if (dedicated_host_mode):
    df_std.reset_index(level=0, drop=True, inplace=True)
    df.reset_index(level=0, drop=True, inplace=True)

In [20]:
#add stds to main columns
if (experiment_type == 'iperf'):
    df['stdUp'] = df_std['bandwidthUpload']
    df['stdDown'] = df_std['bandwitdhDownload']
    df['stdTotal'] = df_std['total']
elif (experiment_type == 'y-cruncher'):
    df['std_computationTime'] = df_std['computationTime']
    df['std_benchmarkTime'] = df_std['benchmarkTime']
    df['std_wallTime'] = df_std['wallTime']
elif (experiment_type == 'sysbench'):
    df['std_wallTime'] = df_std['wallTime']
elif (experiment_type == 'pgbench'):
    df['std_wallTime'] = df_std['wallTime']
    df['std_transactions'] = df_std['transactions']
else:
    print('illegal benchmark name')

In [21]:
df.tail()

,experimentID,bandwidthUpload,bandwitdhDownload,vmId,total,setId,tenants,stdUp,stdDown,stdTotal
instanceID,,,,,,,,,,
i-0dcfa6d3bc470d28d,1573987010,490.5,626.5,30,1117.0,48,48,147.539260,127.224430,147.747532
i-0e167bbfcd6a685f5,1573987010,818.5,661.9,34,1480.4,48,48,417.068406,347.182133,744.610592
i-0ede59d8254436a40,1573987010,558.6,521.0,5,1079.6,48,48,267.392014,176.439477,409.798650
i-0f285214d5c41327e,1573987010,567.5,471.9,21,1039.4,48,48,169.498115,99.217438,248.461130
i-0f472084ac3992ad9,1573987010,735.0,767.6,36,1502.6,48,48,465.934187,414.335130,769.051393


In [28]:
if (experiment_type == 'sysbench' or experiment_type == 'y-cruncher'):
    col = ['setId', 'tenants', 'wallTime', 'std_wallTime']
elif (experiment_type == 'iperf'):
    col = ['bandwidthUpload','bandwitdhDownload','total','setId','tenants','stdUp','stdDown','stdTotal']
elif (experiment_type == 'pgbench'):
    col = ['setId', 'tenants', 'transactions', 'std_transactions']

In [29]:
df = df[col]

In [30]:
df.tail()

,bandwidthUpload,bandwitdhDownload,total,setId,tenants,stdUp,stdDown,stdTotal
instanceID,,,,,,,,
i-0dcfa6d3bc470d28d,490.5,626.5,1117.0,48,48,147.539260,127.224430,147.747532
i-0e167bbfcd6a685f5,818.5,661.9,1480.4,48,48,417.068406,347.182133,744.610592
i-0ede59d8254436a40,558.6,521.0,1079.6,48,48,267.392014,176.439477,409.798650
i-0f285214d5c41327e,567.5,471.9,1039.4,48,48,169.498115,99.217438,248.461130
i-0f472084ac3992ad9,735.0,767.6,1502.6,48,48,465.934187,414.335130,769.051393


In [31]:
df.describe()

,bandwidthUpload,bandwitdhDownload,total,setId,tenants,stdUp,stdDown,stdTotal
count,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000,1176.000000
mean,986.157313,923.810204,1909.967517,32.333333,32.333333,370.608200,320.527657,594.868946
std,848.551042,667.950610,1489.369704,11.430952,11.430952,286.359817,281.612684,430.885771
min,321.300000,423.300000,814.300000,1.000000,1.000000,0.000000,20.630075,47.675873
25%,601.650000,617.275000,1239.475000,24.000000,24.000000,183.894492,165.714303,310.585876
50%,747.950000,740.050000,1493.900000,34.000000,34.000000,298.156397,214.994676,452.681249
75%,898.200000,888.150000,1809.650000,42.000000,42.000000,454.180402,340.557794,715.637879
max,4970.000000,4940.000000,9910.000000,48.000000,48.000000,1972.820812,1601.721588,2611.670891


In [32]:
#print to csv
df.to_csv(path_or_buf='./data/summary/dedicated_host/' + data_name + '-Summary.csv')

In [27]:
#graph the probablity of getting "better vm"